In [5]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely.geometry import box
from shapely.geometry import LineString
from shapely.geometry import MultiLineString
from shapely.ops import unary_union
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import rasterio
from rasterio.mask import mask

# ------------------------------------------------------------
# 1. Load Block Group Polygons from the Census TIGER/Line API
# ------------------------------------------------------------

# The block groups are in Allegheny County, PA (FIPS 42003)
# We download the full county and filter to the target list

TARGET_BGS = ["1500000US420034980001", "1500000US420034980002", "1500000US420034993001", "1500000US420034993002", "1500000US420034994001",
              "1500000US420034994002", "1500000US420034994003", "1500000US420035003001", "1500000US420035003002", "1500000US420035003003",
              "1500000US420035003004", "1500000US420035010001", "1500000US420035030021", "1500000US420035030022", "1500000US420035030023",
              "1500000US420035030024", "1500000US420035030025", "1500000US420035041001", "1500000US420035041002", "1500000US420035041003",
              "1500000US420035041004", "1500000US420035041005", "1500000US420035070001", "1500000US420035070002", "1500000US420035080001",
              "1500000US420035080002", "1500000US420035094001", "1500000US420035094002", "1500000US420035094003", "1500000US420035094004",
              "1500000US420035094005", "1500000US420035100001", "1500000US420035100002", "1500000US420035120001", "1500000US420035120002",
              "1500000US420035130001", "1500000US420035130002", "1500000US420035138001", "1500000US420035138002", "1500000US420035140001",
              "1500000US420035140002", "1500000US420035151001", "1500000US420035151002", "1500000US420035151003", "1500000US420035152001",
              "1500000US420035152002", "1500000US420035153001", "1500000US420035153002", "1500000US420035154011", "1500000US420035154012",
              "1500000US420035154013", "1500000US420035161001", "1500000US420035162001", "1500000US420035162002", "1500000US420035170001",
              "1500000US420035170002", "1500000US420035180011", "1500000US420035180012", "1500000US420035180013", "1500000US420035190001",
              "1500000US420035190002", "1500000US420035190003", "1500000US420035200011", "1500000US420035200012", "1500000US420035200021",
              "1500000US420035200022", "1500000US420035200023", "1500000US420035212001", "1500000US420035212002", "1500000US420035212003",
              "1500000US420035213011", "1500000US420035213012", "1500000US420035213013", "1500000US420035213014", "1500000US420035213021",
              "1500000US420035213022", "1500000US420035213023", "1500000US420035213024", "1500000US420035214011", "1500000US420035214012",
              "1500000US420035214021", "1500000US420035214022", "1500000US420035214023", "1500000US420035220001", "1500000US420035220002",
              "1500000US420035220003", "1500000US420035509001", "1500000US420035509002", "1500000US420035512001", "1500000US420035512002",
              "1500000US420035512003", "1500000US420035513001", "1500000US420035513002", "1500000US420035519001", "1500000US420035520001",
              "1500000US420035520002", "1500000US420035520003", "1500000US420035520004", "1500000US420035521001", "1500000US420035521002",
              "1500000US420035522001", "1500000US420035523001", "1500000US420035523002", "1500000US420035523003", "1500000US420035524001",
              "1500000US420035524002", "1500000US420035524003", "1500000US420035614001", "1500000US420035614002", "1500000US420035614003",
              "1500000US420035614004", "1500000US420035615001", "1500000US420035615002", "1500000US420035639001", "1500000US420035639002",
              "1500000US420035639003", "1500000US420035639004", "1500000US420035642001", "1500000US420035642002", "1500000US420035642003",
              "1500000US420035644001", "1500000US420035644002", "1500000US420035644003", "1500000US420035644004", "1500000US420035644005",
              "1500000US420035644006", "1500000US420035644007", "1500000US420035647001", "1500000US420035647002", "1500000US420035647003"]

# Download Allegheny County block groups
bg_url = "./2023_shape_files/census_shape_block_group.zip"
block_groups = gpd.read_file(bg_url)

# Filter to the specific block groups
bg_selected = block_groups[block_groups["GEOID"].isin([bg[-12:] for bg in TARGET_BGS])]
bg_selected = bg_selected.to_crs(4326)
print("Loaded block groups:", len(bg_selected))

# ------------------------------------------------------------
# Add block-group mean elevation from local SRTM raster
# ------------------------------------------------------------
dem_path = "./2023_shape_files/n40_w080_1arc_v3.tif"
dem = rasterio.open(dem_path)

def polygon_mean_elevation(geom):
    if geom is None or geom.is_empty:
        return np.nan
    geom_dem = gpd.GeoSeries([geom], crs=bg_selected.crs).to_crs(dem.crs).iloc[0]
    out_image, _ = mask(dem, [geom_dem.__geo_interface__], crop=True, filled=False)
    band = out_image[0]
    valid = band[~band.mask]
    if valid.size == 0:
        return np.nan
    return float(valid.mean())

bg_selected["mean_elevation"] = bg_selected.geometry.apply(polygon_mean_elevation)

# ------------------------------------------------------------
# 2. Download road network only within the true BG geometry
# ------------------------------------------------------------

# Use the exact spatial union of your selected block groups
union_polygon = unary_union(bg_selected.geometry)

# Download only the road network inside the real polygon shape
G = ox.graph_from_polygon(union_polygon, network_type="drive", simplify=True)
print("Downloaded road network with", len(G.nodes), "nodes and", len(G.edges), "edges.")

# Convert to GeoDataFrame (edges only)
roads = ox.graph_to_gdfs(G, nodes=False, edges=True)

# Ensure CRS is WGS84
roads = roads.to_crs(4326)

# ------------------------------------------------------------
# 3. Add elevation and slope (grade)
# ------------------------------------------------------------
# NOTE: Requires an API key for elevation data if using Google's service.
# If you don't have one, comment out the next two lines.
# G = ox.add_node_elevations(G, api_key="YOUR_API_KEY")
# G = ox.add_edge_grades(G)

# Convert updated graph back to GeoDataFrame
# roads = ox.graph_to_gdfs(G, nodes=False, edges=True)

# ------------------------------------------------------------
# 4. Compute curvature
# ------------------------------------------------------------
import numpy as np
from shapely.geometry import LineString

import numpy as np
from shapely.geometry import LineString

def compute_curvature_metric(linestring):
    """Curvature = (sum turning angles) / (segment length) in METERS."""
    if not isinstance(linestring, LineString):
        return np.nan
    
    coords = list(linestring.coords)
    if len(coords) < 3:
        return 0.0

    # Convert to UTM for proper metric distances
    ls_utm = gpd.GeoSeries([linestring], crs="EPSG:4326").to_crs("EPSG:26917").iloc[0]
    coords = list(ls_utm.coords)

    total_angle = 0.0

    for i in range(1, len(coords) - 1):
        p1 = np.array(coords[i - 1])
        p2 = np.array(coords[i])
        p3 = np.array(coords[i + 1])

        v1 = p1 - p2
        v2 = p3 - p2

        # Compute turning angle
        dot = np.dot(v1, v2)
        det = np.cross(v1, v2)
        angle = np.arctan2(np.linalg.norm(det), dot)

        total_angle += angle

    length = ls_utm.length
    if length == 0:
        return 0.0

    return total_angle / length

roads["curvature"] = roads.geometry.apply(compute_curvature_metric)


# ------------------------------------------------------------
# 5. Clean up other OSM features
# ------------------------------------------------------------
def safe_int(val, default=1):
    try:
        return int(str(val).split(";")[0])
    except:
        return default

# Handle lanes
if "lanes" in roads.columns:
    roads["lanes"] = roads["lanes"].apply(safe_int)
else:
    roads["lanes"] = 1

def clean_maxspeed(val):
    """
    Normalize OSM maxspeed tags into a clean string or None.
    Handles lists, arrays, strings, and other weird formats.
    """
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return None
    
    # If it's a list or array, take the first valid entry
    if isinstance(val, (list, np.ndarray)):
        # remove None/nan elements
        val = [v for v in val if isinstance(v, (str, int, float))]
        if len(val) == 0:
            return None
        val = val[0]   # take first entry

    # Convert to string
    val = str(val)

    # Strip mph/kmh text
    for unit in [" mph", " km/h", "kmh"]:
        val = val.replace(unit, "")

    # Keep only numbers
    digits = "".join(ch for ch in val if ch.isdigit())
    return digits if digits != "" else None
roads["clean_maxspeed"] = roads["maxspeed"].apply(clean_maxspeed)

def infer_speed(row):
    # 1. If clean maxspeed exists, use it
    if row["clean_maxspeed"] is not None:
        try:
            return int(row["clean_maxspeed"])
        except:
            pass

    # 2. Infer from highway classification
    hwy = str(row["highway"])

    if "residential" in hwy:
        return 25
    if "primary" in hwy:
        return 45
    if "secondary" in hwy:
        return 35
    if "tertiary" in hwy:
        return 30
    if "service" in hwy:
        return 15
    if "motorway" in hwy:
        return 65

    # 3. Fallback
    return 25
roads["speed"] = roads.apply(infer_speed, axis=1)

# Handle bridge
if "bridge" in roads.columns:
    roads["is_bridge"] = roads["bridge"].notnull().astype(int)
else:
    roads["is_bridge"] = 0

# Handle tunnel
if "tunnel" in roads.columns:
    roads["is_tunnel"] = roads["tunnel"].notnull().astype(int)
else:
    roads["is_tunnel"] = 0

# Road type
if "highway" in roads.columns:
    roads["road_type"] = roads["highway"].astype(str)
else:
    roads["road_type"] = "unknown"

# Road surface quality
if "surface" in roads.columns:
    roads["surface"] = roads["surface"].astype(str)
else:
    roads["surface"] = "unknown"

# ------------------------------------------------------------
# 6. Load railway geometry for rail presence and distance-to-rail feature
# ------------------------------------------------------------
from shapely.strtree import STRtree

# -----------------------
# Load TIGER Rails (local)
# -----------------------
rail = gpd.read_file("./2023_shape_files/tl_2023_us_rails.zip")
rail = rail.to_crs(4326)

# -----------------------
# Clip to area of interest
# -----------------------
minx, miny, maxx, maxy = bg_selected.total_bounds
rail_clipped = rail.cx[minx:maxx, miny:maxy]

# ============================================================
# OPTION A — Binary feature: rail_present
# ============================================================
def rail_present(poly, rail_gdf):
    return int(rail_gdf.intersects(poly).any())

bg_selected["rail_present"] = bg_selected.geometry.apply(
    lambda g: rail_present(g, rail_clipped)
)

# ============================================================
# OPTION B — Continuous feature: distance to nearest rail
# ============================================================
# Build geometries list and R-tree index
metric_crs = "EPSG:26917"  # NAD83 / UTM zone 17N; distance is in meters
roads_metric = roads.to_crs(metric_crs)
rail_metric = rail_clipped.to_crs(metric_crs)
rail_geoms = list(rail_metric.geometry)
rail_tree = STRtree(rail_geoms)

METERS_PER_MILE = 1609.34

def distance_to_nearest_rail(geom):
    if geom is None or geom.is_empty:
        return np.nan
    idx = rail_tree.nearest(geom)
    nearest_geom = rail_geoms[idx]
    return geom.distance(nearest_geom) / METERS_PER_MILE

roads["dist_to_rail"] = roads_metric.geometry.apply(distance_to_nearest_rail)

# ------------------------------------------------------------
# 6b. Compute road grade from DEM (rise/run)
# ------------------------------------------------------------
def compute_edge_grade(row):
    geom = row.geometry
    if geom is None or geom.is_empty:
        return np.nan
    geom_dem = gpd.GeoSeries([geom], crs=roads.crs).to_crs(dem.crs).iloc[0]
    if isinstance(geom_dem, MultiLineString):
        geom_dem = max(geom_dem.geoms, key=lambda g: g.length)
    if not isinstance(geom_dem, LineString):
        return np.nan
    coords = list(geom_dem.coords)
    if len(coords) < 2:
        return 0.0
    start = (coords[0][0], coords[0][1])
    end = (coords[-1][0], coords[-1][1])
    samples = list(dem.sample([start, end]))
    elevs = []
    for sample in samples:
        val = sample[0]
        if dem.nodata is not None and val == dem.nodata:
            elevs.append(np.nan)
        else:
            elevs.append(val)
    if len(elevs) < 2 or any(np.isnan(e) for e in elevs):
        return np.nan
    metric_geom = roads_metric.loc[row.name].geometry
    length_m = metric_geom.length
    if length_m == 0:
        return 0.0
    return abs(elevs[1] - elevs[0]) / length_m

roads["grade"] = roads.apply(compute_edge_grade, axis=1)


# Aggregate per block group
roads_in_bg = gpd.sjoin(roads, bg_selected, predicate='intersects')
rail_dist_agg = roads_in_bg.groupby("GEOID")["dist_to_rail"].min().reset_index()

print("Rail-present feature:\n", bg_selected[["GEOID", "rail_present"]])
print("Minimum distance-to-rail per BG:\n", rail_dist_agg)


# ------------------------------------------------------------
# 7. Spatial join roads to block groups
# ------------------------------------------------------------
roads_in_bg = gpd.sjoin(roads, bg_selected, predicate='intersects')

# Map road types to difficulty ranks
road_difficulty = {
    "busway": 0,
    "motorway": 1,
    "trunk": 2,
    "primary": 3,
    "secondary": 4,
    "tertiary": 5,
    "residential": 6,
    "unclassified": 7,
}

roads_in_bg["road_difficulty"] = roads_in_bg["road_type"].map(road_difficulty).fillna(7)

def most_difficult_road_type(series):
    unique = series.unique()
    return max(unique, key=lambda rt: road_difficulty.get(rt, 7))

agg = roads_in_bg.groupby("GEOID").agg({
    "curvature": "mean",
    "lanes": "mean",
    "speed": "mean",
    "is_bridge": "max",
    "is_tunnel": "max",
    "rail_present": "max",
    "dist_to_rail": "min",
    "grade": "mean",
    "surface": lambda x: x.mode()[0] if len(x.mode()) > 0 else None,
    "road_type": most_difficult_road_type,
})


agg.reset_index(inplace=True)

# ------------------------------------------------------------
# 9. Attach land use (optional using OSM landuse tags)
# ------------------------------------------------------------
# bounding box (north, south, east, west)
bbox = (maxy, miny, maxx, minx)
# landuse = ox.features_from_bbox(bbox, {"landuse": True})
# landuse = landuse.to_crs(4326)

# # Example: compute dominant land use in each block group
# def dominant_landuse(poly, landuse_gdf):
#     intersects = landuse_gdf[landuse_gdf.intersects(poly)]
#     if len(intersects) == 0:
#         return None
#     return intersects["landuse"].mode()[0]

# bg_selected["landuse"] = bg_selected.geometry.apply(lambda g: dominant_landuse(g, landuse))

# merge with ODD features
odd_features = agg.merge(bg_selected[["GEOID", "mean_elevation"]], on="GEOID", how="left")

# Append unit info to each column name
units_map = {
    "GEOID": "GEOID [id]",
    "curvature": "curvature [rad/m]",
    "lanes": "lanes [count]",
    "speed": "speed [mph]",
    "is_bridge": "is_bridge [binary]",
    "is_tunnel": "is_tunnel [binary]",
    "rail_present": "rail_present [binary]",
    "dist_to_rail": "dist_to_rail [miles]",
    "grade": "grade [unitless]",
    "mean_elevation": "mean_elevation [meters]",
    "surface": "surface [category]",
    "road_type": "road_type [category]",
}
odd_features = odd_features.rename(columns=units_map)

# ------------------------------------------------------------
# 10. Final ODD dataset
# ------------------------------------------------------------
print("ODD feature table:")
print(odd_features)


Loaded block groups: 130
Downloaded road network with 5625 nodes and 14487 edges.
Rail-present feature:
              GEOID  rail_present
813   420035639003             0
1101  420035644004             1
1102  420035213024             1
1103  420035639001             0
1193  420035614004             0
...            ...           ...
9336  420035647001             0
9369  420035130002             1
9371  420035130001             1
9407  420035647003             1
9408  420035647002             0

[130 rows x 2 columns]
Minimum distance-to-rail per BG:
             GEOID  dist_to_rail
0    420034980001      0.084511
1    420034980002      0.054225
2    420034993001      0.000000
3    420034993002      0.007941
4    420034994001      0.000000
..            ...           ...
124  420035644005      0.882435
125  420035644006      0.000000
126  420035647001      0.240450
127  420035647002      0.000000
128  420035647003      0.004220

[129 rows x 2 columns]
ODD feature table:
       GEOID [

In [7]:
roads["road_type"].apply(type).value_counts()
odd_features["road_type [category]"].unique()

array(['residential', 'unclassified', "['unclassified', 'residential']",
       "['tertiary', 'residential']", 'primary_link', 'trunk_link',
       'tertiary_link', 'motorway_link', 'secondary_link',
       "['secondary', 'primary']"], dtype=object)

In [4]:
from pathlib import Path

output_path = Path("data") / "odd_features.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)

odd_features.to_csv(output_path, index=False)
print(f"Saved ODD features to {output_path}")


Saved ODD features to data/odd_features.csv


In [14]:
road_type_counts = roads["road_type"].value_counts().sort_index()
print("Highway/road_type categories:")
for name, count in road_type_counts.items():
    print(f"{name}: {count}")

Highway/road_type categories:
['secondary', 'primary']: 2
['tertiary', 'primary_link']: 1
['tertiary', 'residential']: 6
['trunk_link', 'primary_link']: 1
['unclassified', 'residential']: 23
busway: 2
motorway: 21
motorway_link: 48
primary: 651
primary_link: 48
residential: 10855
secondary: 492
secondary_link: 8
tertiary: 1761
tertiary_link: 22
trunk: 237
trunk_link: 33
unclassified: 276


In [18]:
TARGET_BGS = ["1500000US420034980001", "1500000US420034980002", "1500000US420034993001", "1500000US420034993002", "1500000US420034994001",
              "1500000US420034994002", "1500000US420034994003", "1500000US420035003001", "1500000US420035003002", "1500000US420035003003",
              "1500000US420035003004", "1500000US420035010001", "1500000US420035030021", "1500000US420035030022", "1500000US420035030023",
              "1500000US420035030024", "1500000US420035030025", "1500000US420035041001", "1500000US420035041002", "1500000US420035041003",
              "1500000US420035041004", "1500000US420035041005", "1500000US420035070001", "1500000US420035070002", "1500000US420035080001",
              "1500000US420035080002", "1500000US420035094001", "1500000US420035094002", "1500000US420035094003", "1500000US420035094004",
              "1500000US420035094005", "1500000US420035100001", "1500000US420035100002", "1500000US420035120001", "1500000US420035120002",
              "1500000US420035130001", "1500000US420035130002", "1500000US420035138001", "1500000US420035138002", "1500000US420035140001",
              "1500000US420035140002", "1500000US420035151001", "1500000US420035151002", "1500000US420035151003", "1500000US420035152001",
              "1500000US420035152002", "1500000US420035153001", "1500000US420035153002", "1500000US420035154011", "1500000US420035154012",
              "1500000US420035154013", "1500000US420035161001", "1500000US420035162001", "1500000US420035162002", "1500000US420035170001",
              "1500000US420035170002", "1500000US420035180011", "1500000US420035180012", "1500000US420035180013", "1500000US420035190001",
              "1500000US420035190002", "1500000US420035190003", "1500000US420035200011", "1500000US420035200012", "1500000US420035200021",
              "1500000US420035200022", "1500000US420035200023", "1500000US420035212001", "1500000US420035212002", "1500000US420035212003",
              "1500000US420035213011", "1500000US420035213012", "1500000US420035213013", "1500000US420035213014", "1500000US420035213021",
              "1500000US420035213022", "1500000US420035213023", "1500000US420035213024", "1500000US420035214011", "1500000US420035214012",
              "1500000US420035214021", "1500000US420035214022", "1500000US420035214023", "1500000US420035220001", "1500000US420035220002",
              "1500000US420035220003", "1500000US420035509001", "1500000US420035509002", "1500000US420035512001", "1500000US420035512002",
              "1500000US420035512003", "1500000US420035513001", "1500000US420035513002", "1500000US420035519001", "1500000US420035520001",
              "1500000US420035520002", "1500000US420035520003", "1500000US420035520004", "1500000US420035521001", "1500000US420035521002",
              "1500000US420035522001", "1500000US420035523001", "1500000US420035523002", "1500000US420035523003", "1500000US420035524001",
              "1500000US420035524002", "1500000US420035524003", "1500000US420035614001", "1500000US420035614002", "1500000US420035614003",
              "1500000US420035614004", "1500000US420035615001", "1500000US420035615002", "1500000US420035639001", "1500000US420035639002",
              "1500000US420035639003", "1500000US420035639004", "1500000US420035642001", "1500000US420035642002", "1500000US420035642003",
              "1500000US420035644001", "1500000US420035644002", "1500000US420035644003", "1500000US420035644004", "1500000US420035644005",
              "1500000US420035644006", "1500000US420035644007", "1500000US420035647001", "1500000US420035647002", "1500000US420035647003"]

# Download Allegheny County block groups
bg_url = "./2023_shape_files/census_shape_block_group.zip"
block_groups = gpd.read_file(bg_url)

# Filter to the specific block groups
bg_selected = block_groups[block_groups["GEOID"].isin([bg[-12:] for bg in TARGET_BGS])]
bg_selected = bg_selected.to_crs(4326)
print("Loaded block groups:", len(bg_selected))

# ------------------------------------------------------------
# Add block-group mean elevation from local SRTM raster
# ------------------------------------------------------------
dem_path = "./2023_shape_files/n40_w080_1arc_v3.tif"
dem = rasterio.open(dem_path)

def polygon_mean_elevation(geom):
    if geom is None or geom.is_empty:
        return np.nan
    geom_dem = gpd.GeoSeries([geom], crs=bg_selected.crs).to_crs(dem.crs).iloc[0]
    out_image, _ = mask(dem, [geom_dem.__geo_interface__], crop=True, filled=False)
    band = out_image[0]
    valid = band[~band.mask]
    if valid.size == 0:
        return np.nan
    return float(valid.mean())

bg_selected["mean_elevation"] = bg_selected.geometry.apply(polygon_mean_elevation)

# ------------------------------------------------------------
# 2. Download road network only within the true BG geometry
# ------------------------------------------------------------

# Use the exact spatial union of your selected block groups
union_polygon = unary_union(bg_selected.geometry)

# Download only the road network inside the real polygon shape
G = ox.graph_from_polygon(union_polygon, network_type="drive", simplify=True)
print("Downloaded road network with", len(G.nodes), "nodes and", len(G.edges), "edges.")

# Convert to GeoDataFrame (edges only)
roads = ox.graph_to_gdfs(G, nodes=False, edges=True)

# Ensure CRS is WGS84
roads = roads.to_crs(4326)

Loaded block groups: 130
Downloaded road network with 5625 nodes and 14487 edges.


In [28]:
road_type_categories = set()
for road_type in roads['highway'].values:
    if isinstance(road_type, list):
        for rt in road_type:
            road_type_categories.add(rt)
    else:
        road_type_categories.add(road_type)
road_type_categories

{'busway',
 'motorway',
 'motorway_link',
 'primary',
 'primary_link',
 'residential',
 'secondary',
 'secondary_link',
 'tertiary',
 'tertiary_link',
 'trunk',
 'trunk_link',
 'unclassified'}